In [1]:
import re
from rdkit import Chem
import pandas as pd
import numpy as np
from molmass import Formula
from matplotlib import pyplot as plt
from pyimzml.ImzMLParser import ImzMLParser
# from pyimzml.ImzMLParser import getionimage

In [2]:
slides = pd.read_csv('2020_Feb_21_met_slides_v1.csv', sep='\t')

In [3]:
slides = slides.dropna(subset=['formula'])

In [4]:
slides['exact'] = slides['formula'].apply(lambda x: Formula(x).isotope.mass)

In [5]:
adduct_dict = {'m_+H+': 1.0073, 'm_+Na+': 22.9898, 'm_+K+': 38.9631, 'm_-H+': -1.0073}

In [6]:
for adduct, dmass in adduct_dict.items():
    slides[adduct] = slides['exact'].apply(lambda x: x + dmass)

In [7]:
slides = slides.drop(columns=['m+H+','m+Na+','M+K+','m-H+'])

In [31]:
slide = slides[slides.D == 1]

In [32]:
slide

,s_neg,s_pos,A,B,C,D,n384,p384,R,C.1,...,formula,smiles,cas,rdkit,z,exact,m_+H+,m_+Na+,m_+K+,m_-H+
230,20,12,NaN,NaN,NaN,1.0,90,1,D,18,...,C4H9NO2,NCCCC(O)=O,56-12-2,NaN,NaN,103.063329,104.070629,126.053129,142.026429,102.056029
231,20,12,NaN,NaN,NaN,1.0,91,1,D,19,...,C16H34O,CCCCCCCCCCCCCCCCO,36653-82-4,.,NaN,242.260966,243.268266,265.250766,281.224066,241.253666
232,20,12,NaN,NaN,NaN,1.0,92,1,D,20,...,C4H9NO2,CC(CN)C(O)=O,144-90-1,.,NaN,103.063329,104.070629,126.053129,142.026429,102.056029
233,20,12,NaN,NaN,NaN,1.0,93,1,D,21,...,C9H10N2,CC1=CC2=C(C=C1C)N=CN2,582-60-5,.,NaN,146.084398,147.091698,169.074198,185.047498,145.077098
234,20,12,NaN,NaN,NaN,1.0,94,1,D,22,...,C4H9NO2,CC(C)(N)C(O)=O,62-57-7,.,NaN,103.063329,104.070629,126.053129,142.026429,102.056029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,32,28,NaN,NaN,NaN,1.0,551,2,G,23,...,C10H13N5O5,NC1=NC2=C(N=CN2[C@@H]2O[C@H](CO)[C@@H](O)[C@H]...,118-00-3,.,NaN,283.091669,284.098969,306.081469,322.054769,282.084369
442,32,28,NaN,NaN,NaN,1.0,552,2,G,24,...,C27H33N9O15P2,CC1=CC2=C(C=C1C)N(C[C@H](O)[C@H](O)[C@H](O)COP...,84366-81-4,.,NaN,785.157135,786.164435,808.146935,824.120235,784.149835
444,32,28,NaN,NaN,NaN,1.0,571,2,H,19,...,C18H10N2O4,CC1=C2NC=C3C2=C(C2=CNC4=C(C)C(=O)C(=O)C3=C24)C...,8049-97-6,.,NaN,318.064057,319.071357,341.053857,357.027157,317.056757
446,32,28,NaN,NaN,NaN,1.0,573,2,H,21,...,C10H15N2O8P,CC1=CN([C@H]2C[C@H](O)[C@@H](COP(O)(O)=O)O2)C(...,33430-62-5,.,NaN,322.056602,323.063902,345.046402,361.019702,321.049302


In [33]:
export = list(slide)[-4:]

In [34]:
export

['m_+H+', 'm_+Na+', 'm_+K+', 'm_-H+']

In [35]:
H_plus = ",".join([f'{x:.4f}' for x in slide[export[0]].unique()])
Na_plus = ",".join([f'{x:.4f}' for x in slide[export[1]].unique()])
K_plus = ",".join([f'{x:.4f}' for x in slide[export[2]].unique()])
H_minus = ",".join([f'{x:.4f}' for x in slide[export[3]].unique()])

In [36]:
#out = H_plus + Na_plus + K_plus
out = H_minus

In [37]:
out

'102.0560,241.2537,145.0771,299.2016,103.0037,105.0193,187.0400,690.5079,105.0346,163.0400,106.0662,112.0516,429.3738,113.0356,301.2384,114.0196,353.3425,114.0560,116.0717,120.0124,121.0295,157.0255,124.0516,157.0367,124.0880,159.0663,125.0356,159.0775,162.0230,130.0509,162.0421,163.0261,163.0612,166.9751,131.0350,167.0210,167.0350,169.0506,134.0281,171.0064,134.0472,171.0299,174.0408,138.0196,174.0520,138.9802,174.0560,139.9754,444.1637,223.0724,445.0531,225.0517,448.3068,225.0993,464.3017,505.9884,242.0782,243.0622,506.9725,243.2190,521.9834,581.2405,259.0224,583.2562,588.0749,604.0699,743.0522,275.0173,744.0838,280.1051,766.1079,282.0844,784.1498,317.0568,321.0493'

### Let's look for target masses in our files!

In [38]:
def getionimage(p, mz_value, tol=0.1, z=1, reduce_func=sum):
    from pyimzml.ImzMLParser import _bisect_spectrum
    tol = abs(tol)
    x0, y0 = np.min(p.coordinates, axis=0)[:2]
    width, height = np.max(p.coordinates, axis=0)[:2] - [x0, y0] + [1, 1]
    im = np.zeros((height, width))
    for i, (x, y, z_) in enumerate(p.coordinates):
            mzs, ints = p.getspectrum(i)
            min_i, max_i = _bisect_spectrum(mzs, mz_value, tol)
            im[y - y0, x - x0] = reduce_func(ints[min_i:max_i+1])
    return im

In [39]:
def ppm_5(mz):
    # Calculates 5 ppm from input mass
    mz_window = mz * 5 / 1000000
    return mz_window

def ims_plot(ims_file, mz1, mz2, mz3, title):
    ims = ImzMLParser(ims_file, parse_lib='ElementTree')
    img1 = getionimage(ims, mz1, tol=ppm_5(mz1), z=1, reduce_func=sum)
    img2 = getionimage(ims, mz2, tol=ppm_5(mz2), z=1, reduce_func=sum)
    img3 = getionimage(ims, mz3, tol=ppm_5(mz3), z=1, reduce_func=sum)
   
    img_sum = img1 + img2 + img3
    
    f, axarr = plt.subplots(2,2)
    axarr[0,0].imshow(img1)
    axarr[0,1].imshow(img2)
    axarr[1,0].imshow(img3)
    axarr[1,1].imshow(img_sum)
    
    f.suptitle(title)
    f.savefig('ims_plots/' + title)
    plt.close()
    
    return

In [20]:
ims_plot(ims_file, 100.1, 200, 300, 'Wow')

In [40]:
def ims_plotter(r, polarity, which_slide, ims_file):
    title = str(which_slide) + '_p384_' + str(r.R) + str(r['C.1']) + '_cmpd' + str(r.cmpd)  
    if polarity == 'p':
        title = 'slide_' + str(r.s_pos) + title
        img = ims_plot(ims_file,
                       r['m_+H+'],
                       r['m_+Na+'],
                       r['m_+K+'],
                       title)
        
    elif polarity == 'n':
        title = 'slide_' + str(r.s_neg) + title
        img = ims_plot(ims_file,
                       r['m_-H+'],
                       r['m_-H+'],
                       r['m_-H+'],
                       title)
        
    return title

In [52]:
#ims_file = 'Example_Processed.imzML'
imss = ['/Volumes/alexandr/baxter/2020_Feb_26_CMBR_slides_17_20_neg/2020_Feb_25_slide_18n.imzML',
        '/Volumes/alexandr/baxter/2020_Feb_26_CMBR_slides_17_20_neg/2020_Feb_25_slide_19n.imzML',
        '/Volumes/alexandr/baxter/2020_Feb_26_CMBR_slides_17_20_neg/2020_Feb_25_slide_20n.imzML']

ims_file = imss[2]

In [53]:
which_slide = 'D' # A, B, C, D
which_plate = 1 # 1, 2
slide = slides[(slides[which_slide] == 1) &
              (slides['p384'] == which_plate)].copy(deep=True)
polarity = 'n' # 'n'
slide['ims_plot'] = slide.apply(lambda row: ims_plotter(row, 
                                                        polarity,
                                                        which_slide,
                                                        ims_file), axis =1)

In [39]:
slide.head(3)

,s_neg,s_pos,A,B,C,D,n384,p384,R,C.1,...,smiles,cas,rdkit,z,exact,m_+H+,m_+Na+,m_+K+,m_-H+,ims_plot
230,20,12,NaN,NaN,NaN,1.0,90,1,D,18,...,NCCCC(O)=O,56-12-2,NaN,NaN,103.063329,104.070629,126.053129,142.026429,102.056029,slide_12D_p384_D18_cmpdGAMMA-AMINOBUTYRATE
231,20,12,NaN,NaN,NaN,1.0,91,1,D,19,...,CCCCCCCCCCCCCCCCO,36653-82-4,.,NaN,242.260966,243.268266,265.250766,281.224066,241.253666,slide_12D_p384_D19_cmpdHEXADECANOL
232,20,12,NaN,NaN,NaN,1.0,92,1,D,20,...,CC(CN)C(O)=O,144-90-1,.,NaN,103.063329,104.070629,126.053129,142.026429,102.056029,slide_12D_p384_D20_cmpdAMINOISOBUTANOATE


In [30]:
slide.to_csv('slide_10b_p384_1', sep='\t')

In [57]:
ms_annot = pd.read_csv('metaspce_p10.tsv', sep='\t', skiprows=2)

In [59]:
ms_annot.head(3)

,group,datasetName,datasetId,formula,adduct,mz,msm,fdr,rhoSpatial,rhoSpectral,rhoChaos,moleculeNames,moleculeIds,minIntensity,maxIntensity,totalIntensity,offSample,rawOffSampleProb,isomerIons,isobarIons
0,European Molecular Biology Laboratory,20200221_slide_10,2020-02-21_16h54m18s,C21H14O10,M+Na,449.047879,0.979616,0.05,0.990156,0.998311,0.991029,"5,14-bis(acetyloxy)-4-hydroxy-9-oxo-8,17-dioxa...","HMDB0128404, HMDB0128405, HMDB0128406, HMDB012...",0,260042.85940,2.928534e+08,True,0.629278,NaN,"C18H18O11+K+, C15H17N5O6S2+Na+, C16H18O13S+H+"
1,European Molecular Biology Laboratory,20200221_slide_10,2020-02-21_16h54m18s,C20H38O2,M+H,311.294418,0.966337,0.05,0.978853,0.988212,0.998990,"6,8-Icosanedione, 5,7-Icosanedione, 4,6-Icosan...","HMDB0035572, HMDB0035573, HMDB0035574, HMDB000...",0,10060.01563,2.242692e+04,True,0.778934,NaN,NaN
2,European Molecular Biology Laboratory,20200221_slide_10,2020-02-21_16h54m18s,C28H20O14,M+Na,603.074487,0.953334,0.05,0.969220,0.997588,0.985988,Epitheaflavic acid 3'-gallate,HMDB0040610,0,48074.60547,3.279971e+07,True,0.552478,NaN,C25H24O15+K+


In [60]:
metaspace_list = list(ms_annot.formula)

In [62]:
which_slides = ['A', 'B', 'C', 'D']
out_dict = {}

for w in which_slides:
    slide = slides[(slides[which_slide] == 1) & (slides['p384'] == 1)].copy(deep=True)
    intersect = list(set(metaspace_list) & set(slide.formula))
    out_dict[w] = len(intersect)

In [63]:
out_dict

{'A': 3, 'B': 3, 'C': 3, 'D': 3}

Done:
1. Less than 5 matched spots per slide, assuming conservative 1-2 shifts or flipped.
2. Check if slides reversed?  E.g. Slide 12 = 1a.  No better
3. Check Metaspace formulas assigned overalap?
    Slide 10: {'A': 3, 'B': 3, 'C': 3, 'D': 3}
    Slide 12:
4. No-go positive mode (25 ug/mL).

To do:
1. Run slides negative mode.
2. Run GNPS.
3. Drugs